<a href="https://colab.research.google.com/github/Tiziano-Test/Climate-Change-Earth-Surface-Temperature-/blob/main/Climate_Change_Earth_Surface_Temperature_Data_6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
6.3: Geographical Visualizations with Python

Step 1: Upload the Zip File to Colab

In [ ]:
!pip install kaggle
from google.colab import files

# Upload your kaggle.json file here
files.upload()

# Create .kaggle directory and move kaggle.json to it
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle
!kaggle datasets download -d berkeleyearth/climate-change-earth-surface-temperature-data

# Unzip the dataset
!unzip climate-change-earth-surface-temperature-data.zip

Saving Global Land Temperatures By City.zip to Global Land Temperatures By City.zip
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data
License(s): CC-BY-NC-SA-4.0
 90% 76.0M/84.7M [00:00<00:00, 113MB/s]
100% 84.7M/84.7M [00:00<00:00, 108MB/s]
Archive:  climate-change-earth-surface-temperature-data.zip
  inflating: GlobalLandTemperaturesByCity.csv  
  inflating: GlobalLandTemperaturesByCountry.csv  
  inflating: GlobalLandTemperaturesByMajorCity.csv  
  inflating: GlobalLandTemperaturesByState.csv  
  inflating: GlobalTemperatures.csv  


Step 2: Load the CSV Files into DataFrames

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Set seaborn style
sns.set(style="whitegrid")

# Load the CSV files into dataframes
global_temp_by_city = pd.read_csv('GlobalLandTemperaturesByCity.csv')
global_temp_by_country = pd.read_csv('GlobalLandTemperaturesByCountry.csv')
global_temp_by_major_city = pd.read_csv('GlobalLandTemperaturesByMajorCity.csv')
global_temp_by_state = pd.read_csv('GlobalLandTemperaturesByState.csv')
global_temp = pd.read_csv('GlobalTemperatures.csv')

# Display the first few rows of each dataset to understand their structure
print("GlobalLandTemperaturesByCity")
print(global_temp_by_city.head())

print("\nGlobalLandTemperaturesByCountry")
print(global_temp_by_country.head())

print("\nGlobalLandTemperaturesByMajorCity")
print(global_temp_by_major_city.head())

print("\nGlobalLandTemperaturesByState")
print(global_temp_by_state.head())

print("\nGlobalTemperatures")
print(global_temp.head())


GlobalLandTemperaturesByCity
           dt  AverageTemperature  AverageTemperatureUncertainty   City  \
0  1743-11-01               6.068                          1.737  Århus   
1  1743-12-01                 NaN                            NaN  Århus   
2  1744-01-01                 NaN                            NaN  Århus   
3  1744-02-01                 NaN                            NaN  Århus   
4  1744-03-01                 NaN                            NaN  Århus   

   Country Latitude Longitude  
0  Denmark   57.05N    10.33E  
1  Denmark   57.05N    10.33E  
2  Denmark   57.05N    10.33E  
3  Denmark   57.05N    10.33E  
4  Denmark   57.05N    10.33E  

GlobalLandTemperaturesByCountry
           dt  AverageTemperature  AverageTemperatureUncertainty Country
0  1743-11-01               4.384                          2.294   Åland
1  1743-12-01                 NaN                            NaN   Åland
2  1744-01-01                 NaN                            NaN   Åland
3  

Step 3: Import Required Libraries

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import json

# For handling JSON/GeoJSON data
import requests


Step 4: Load and Wrangle Data

I'll focus on the GlobalLandTemperaturesByCountry.csv for the choropleth map, as it provides country-level temperature data.

In [ ]:
# Load the CSV file into a dataframe
global_temp_by_country = pd.read_csv('GlobalLandTemperaturesByCountry.csv')

# Display the first few rows of the dataset to understand its structure
print(global_temp_by_country.head())


           dt  AverageTemperature  AverageTemperatureUncertainty Country
0  1743-11-01               4.384                          2.294   Åland
1  1743-12-01                 NaN                            NaN   Åland
2  1744-01-01                 NaN                            NaN   Åland
3  1744-02-01                 NaN                            NaN   Åland
4  1744-03-01                 NaN                            NaN   Åland


Step 5:

We need to clean the data to remove missing values and calculate average temperatures for last 100 years.

In [ ]:
# Convert 'dt' column to datetime
global_temp_by_country['dt'] = pd.to_datetime(global_temp_by_country['dt'])

# Extract year from the 'dt' column
global_temp_by_country['Year'] = global_temp_by_country['dt'].dt.year

# Filter data for the last 100 years
recent_data = global_temp_by_country[global_temp_by_country['Year'] >= global_temp_by_country['Year'].max() - 100]

# Calculate the average temperature by country for the last 100 years
avg_temp_by_country = recent_data.groupby('Country')['AverageTemperature'].mean().reset_index()

# Rename columns for better understanding
avg_temp_by_country.columns = ['Country', 'AverageTemperature']

print(avg_temp_by_country.head())


          Country  AverageTemperature
0     Afghanistan           14.297799
1          Africa           24.311560
2         Albania           12.944224
3         Algeria           23.344309
4  American Samoa           26.788945


Step 6: Create a Choropleth Map

We'll use folium to create a choropleth map. We'll need a GeoJSON file that contains the geographical boundaries of the countries.

In [ ]:
# Download GeoJSON file with country boundaries
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
response = requests.get(url)
world_geo = response.json()

# Create a base map
m = folium.Map(location=[0, 0], zoom_start=2)

# Create the choropleth map
folium.Choropleth(
    geo_data=world_geo,
    name='choropleth',
    data=avg_temp_by_country,
    columns=['Country', 'AverageTemperature'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Temperature (°C)'
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save the map as an HTML file
map_path = '/content/choropleth_map.html'
m.save(map_path)

# Download the HTML file
files.download(map_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Step 7: Analyze Results and Answer Research Questions

Open the generated choropleth_map.html file to view the map.

The creation of the previous choropleth map partially answers some of the existing research questions and highlights areas for further investigation. Here is an analysis based on the created map and potential new research questions that arise from it:
Existing Research Questions Addressed

    Geographical Analysis:
        Which regions experience the highest average temperatures?
            The choropleth map shows that regions near the equator, particularly in Africa, South America, and Southeast Asia, experience the highest average temperatures. These regions are highlighted in dark red on the map.
        How do temperature variations differ between continents?
            The map indicates that continents such as Africa and South America generally experience higher average temperatures compared to continents like Europe and North America. This visual differentiation provides a clear geographical understanding of temperature distribution across continents.

    Temperature Trends:
        The map does not directly indicate specific cities or countries where temperature changes are more pronounced over time. This requires a detailed trend analysis for each city or country.

New Research Questions

The choropleth map brings up some new questions and areas for deeper investigation:

    Temperature Extremes:
        Why do certain regions like North America and parts of Europe appear to have relatively lower average temperatures despite the ongoing global warming trend?
        Are there specific climatic or geographical factors contributing to these observations?

    Impact on Ecosystems and Populations:
        How do the high average temperatures in equatorial regions affect local ecosystems and human populations?
        Are there observable patterns of migration or changes in biodiversity in these regions as a result of higher temperatures?

    Regional Anomalies:
        Are there any regions that deviate significantly from the expected temperature patterns based on their geographical location? For example, why do some areas in high latitude regions show higher average temperatures than others?

    Urban vs. Rural Temperature Differences:
        How do temperatures in major cities compare to rural areas within the same country or region? Is there a noticeable urban heat island effect?

    Seasonal and Temporal Patterns:
        How do seasonal temperature variations contribute to the overall average temperatures depicted on the map?
        Are there regions where seasonal temperature variations are more extreme compared to others?